In [6]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn'}
FSIZE = [5,4]
FDPI = 100
SAVEFIGS = True

In [7]:
# DMG functions
def encounter_multiplier_DMG(P, N):
    """Returns the encounter multiplier given by the DMG
    P -- integer number of PCs in the encounter
    N -- integer number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(N,1)].argmax()
    if P >= 6:
        i -= 1
    elif P <= 2:
        i += 1
    return m_array[i]

# encounter multiplier approximations that assume all monsters are identical
def encounter_multiplier_approx(P=4, N=1, PD=0, AoE_pct=0, AoE_eff=1.0):
    """Returns the approximate encounter multiplier
    P -- number of PCs in the encounter
    N -- number of NPCs in the encounter
    PD -- number of PCs expected to die during encounter
    AoE_pct -- decimal percent of damage done to each NPC using AoE effects
    AoE_eff -- ratio of AoE DPR per target to single target DPR
    """
    EM = (N + 1)*(1 - AoE_pct)/2 + AoE_pct/AoE_eff
    PD = min(PD, P)
    EM *= 8.0*P/(2*P*P - 2*PD*P + PD*(PD + 1))
    return EM

In [8]:
# XP log functions
def create_XP_log(eHPs, eDPRs, damage):
    """Returns an XP_log
    eHPs -- list of effective hit point values
    eDPRs -- list of effective damage per round values
    damage -- list of damage entries
        damage = [
            {'targets': [1], 'damage': 5, 'efficiency': 1.0, 'color': 'w'},
            {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'w'}
        ]
    """
    nC = len(eDPRs)
    eDPR_mins = [sum(eDPRs[0:i]) for i in range(nC)]
    eHP_mins = [sum(eHPs[0:i]) for i in range(nC)]
    dmgs = [0]*nC
    eHP_dmgs = [0]*nC
    XP_log = []
    for d in damage:
        # determine if the damage affects a single target or multiple
        if len(d['targets']) > 1:
            dmg_type = 'multiple'
        else:
            dmg_type = 'single'

        # determine which target takes the most damage
        for i in range(nC):
            if (i in d['targets']):
                dmgs[i] = min(eHP_dmgs[i] + d['damage'], eHPs[i]) - eHP_dmgs[i]
            else:
                dmgs[i] = 0
        t_max = np.argmax(dmgs)
        dmg_max = dmgs[t_max]

        # construct log entries for damage
        for i in range(nC):
            if eHP_dmgs[i] < eHPs[i]:
                if i not in d['targets']:
                    dmg = dmg_max
                    t = t_max
                else:
                    dmg = dmgs[i]
                    t = i
                
                XP_log.append({
                    'type': dmg_type,
                    'efficiency': d['efficiency'],
                    'target': t,
                    'ally': i,
                    'damage': d['damage'],
                    'XP': 0.25*dmg*eDPRs[i]/(d['efficiency']),
                    'eHP_min': eHP_mins[t] + eHP_dmgs[t],
                    'eDPR_min': eDPR_mins[i],
                    'eHP_delta': dmg,
                    'eDPR_delta': eDPRs[i],
                    'color': d['color']
                })
        
        # update damage taken
        for t in d['targets']:
            eHP_dmgs[t] += dmgs[t]
        
    return XP_log

def generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, **kwargs):
    """Generate XP diagram and returns axes for that diagram
    eHPs -- list of effective hit point values
    eDPRs -- list of effective damage per round values
    XP_log -- list of XP contributions for the encounter
    **kwargs
        fontsize -- font size for the figure's text
        ax -- pyplot axes
    """
    nC = len(eHPs)
    eHP_mins = [sum(eHPs[0:i]) for i in range(len(eHPs))]
    eDPR_mins = [sum(eDPRs[0:i]) for i in range(len(eDPRs))]
    eHP_max = sum(eHPs)
    eDPR_max = sum(eDPRs)

    fontsize = kwargs.get('fontsize', None)
    fig = kwargs.get('fig', go.Figure())
    title = kwargs.get('title', None)

    # initialize figure
    #ax.add_patch(plt.Rectangle((0,0), eDPR_max, eHP_max, fc='white', ec='none', alpha=1.0))

    # apply damage and color XP boxes accordingly
    for x in XP_log:
        fig.add_shape(type="rect",
            x0=x['eDPR_min'], x1=x['eDPR_min']+x['eDPR_delta'], 
            y0=x['eHP_min'],  y1=x['eHP_min']+x['eHP_delta'],
            line=dict(color='rgba(250,250,250,0)'),
            fillcolor=x['color'],
            opacity=0.3,
        )

    # add boarder lines
    def add_reference_line(fig, x, y):
        fig.add_trace(go.Scatter(
            x=x, y=y,
            mode='lines',
            line=dict(color='black', dash='solid'),
            showlegend=False, 
            hoverinfo='skip',
        ))

    for i in range(nC):
        add_reference_line(fig, x=[0, eDPR_max], y=[eHP_mins[i],eHP_mins[i]])
        add_reference_line(fig, x=[eDPR_mins[i],eDPR_mins[i]], y=[0, eHP_max])
    add_reference_line(fig, x=[0, eDPR_max], y=[eHP_max, eHP_max])
    add_reference_line(fig, x=[eDPR_max, eDPR_max], y=[0, eHP_max])

    # create XP box borders
    for i in range(nC):
        for j in range(nC):
            if i == j:
                ls = '-'
            else:
                ls = '-'
            #ax.add_patch(plt.Rectangle((eDPR_mins[i], eHP_mins[j]), eDPRs[i], eHPs[j], fc='none', ec='black', linestyle=ls))

    # create XP box labels
    for i in range(nC):
        for j in range(nC):
            fig.add_annotation(
                x=eDPR_mins[i] + 0.5*eDPRs[i], 
                y=eHP_mins[j] + 0.5*eHPs[j],
                showarrow=False,
                text=f'$\mathit{{XP}}_{{{j+1},{i+1}}}$',
            )

    # create axes labels
    for i in range(nC):
        # x-axis
        fig.add_annotation(
            x=eDPR_mins[i] + 0.5*eDPRs[i], y=-(0.7/16)*eHP_max, 
            showarrow=False,
            text=f'$\mathit{{eDPR}}_{{{i+1}}}$',
        )
        # y-axis
        fig.add_annotation(
            x=-(0.7/16)*eDPR_max, y=(eHP_mins[i] + 0.5*eHPs[i]), 
            showarrow=False,
            text=f'$\mathit{{eHP}}_{{{i+1}}}$',
            textangle=-90,
        )

    return fig

In [9]:
# plots the encounter multiplier vs the number of PCs for three different numbers of NPCs
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='number of PCs',
            range=[0.5,7.5],
            tick0=0, dtick=1,
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,8.1],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

P = np.arange(1, 8)

nList = [1, 7, 15]
cList = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

for i in range(len(nList)):
    n = nList[i]
    color = cList[i]
    y = np.array([encounter_multiplier_DMG(p, n) for p in P ])
    fig.add_trace(go.Scatter(
        x=P, 
        y=y,
        mode='lines+markers', 
        #legendgroup=f'{n:d} NPCs',
        #legendgrouptitle_text=f'{n:d} NPCs',
        name=f'{n:d} NPCs - DMG',
        marker_color=color,
        line_color=color,
        hovertemplate=
            f'<b>DMG {n:d} NPCs,' + 
            ' %{x:d} PCs</b><br>'+
            'multiplier %{y:,.1f}' + 
            '<extra></extra>'
    ))

    y = np.array([encounter_multiplier_approx(p, 1)*encounter_multiplier_DMG(4, n) for p in P ])
    fig.add_trace(go.Scatter(
        x=P, 
        y=y,
        mode='lines+markers', 
        #legendgroup=f'{n:d} NPCs',
        name=f'{n:d} NPCs - approx',
        marker_color=color,
        marker_symbol='square',
        line_color=color,
        line_dash='dash',
        hovertemplate=
            f'<b>approx {n:d} NPCs,' + 
            ' %{x:d} PCs</b><br>'+
            'multiplier %{y:,.1f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-em-vs-pcs-large')
    tfb.save_fig_html(fig, format='small', name='fig-em-vs-pcs-small')

In [10]:
# An example encounter diagram for two NPCs defeated in order using single target damage
eDPRs = [3, 3, 3]
eHPs  = [3, 3, 3]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [1], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [2], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        title=dict(
            text=r'$\mathrm{PC}\ 1 \rightarrow \mathrm{PC}\ 2 \rightarrow \mathrm{PC}\ 3$',
            font_size=16,
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
        ),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-three-pcs-single-target-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-three-pcs-single-target-small', style='aspect-ratio: 600/600;')

In [11]:
# plots the encounter multiplier vs the number of PCs for three different numbers of NPCs

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='number of PCs',
            range=[0.5,7.5],
            tick0=0, dtick=1,
        ),
        yaxis=dict(
            title_text='encounter XP ratio',
            range=[0,2],
            tick0=0, dtick=0.25,
            tickformat = '.2f',
        ),
    )
)

# plot data
P = np.arange(1, 8)
y = np.array([encounter_multiplier_approx(p, 1, PD=p)/encounter_multiplier_approx(p, 1) for p in P ])
fig.add_trace(go.Scatter(
    x=P,
    y=y,
    mode='lines',
    showlegend=False,
    hovertemplate=
        'PCs %{x:d}<br>'+
        'XP ratio %{y:,.2f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-single-target-em-ratio-vs-pcs-large')
    tfb.save_fig_html(fig, format='small', name='fig-single-target-em-ratio-vs-pcs-small')

In [12]:
# An example encounter diagram for two NPCs defeated in order using single target damage
eDPRs = [3, 3, 3, 3]
eHPs  = [3, 3, 3, 3]
eDPR_total = np.sum(eDPRs)
eHP_total = np.sum(eHPs)

damage = [
    {'targets': [0], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [1], 'damage': 3, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [2], 'damage': 2.99, 'efficiency': 1.0, 'color': 'blue'},
    {'targets': [3], 'damage': 2.99, 'efficiency': 1.0, 'color': 'blue'}
]

XP_log = create_XP_log(eHPs, eDPRs, damage)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=15, r=15, b=15, t=15, pad=4),
        title=dict(
            text=r'$\mathrm{PC}\ 1 \rightarrow \mathrm{PC}\ 2$',
            font_size=16,
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
        ),
        #margin=dict(autoexpand=True),
        dragmode=False,
        xaxis=dict(
            range=[-0.1*eDPR_total, 1.1*eDPR_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
        yaxis=dict(
            range=[-0.1*eHP_total, 1.1*eHP_total],
            #autorange=True,
            mirror=False,
            showgrid=False,
            showline=False,
            zeroline=False,
            showticklabels=False,
            ticks='', minor=dict(ticks=''),
        ),
    )
)

fig = generate_XP_diagram_plotly(eHPs, eDPRs, XP_log, fig=fig)

# show figure
fig.update_layout(width=600, height=600)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-xp-encounter-diagram-pcs-single-target-partial-large', style='aspect-ratio: 600/600;')
    tfb.save_fig_html(fig, format='small', name='fig-xp-encounter-diagram-pcs-single-target-partial-small', style='aspect-ratio: 600/600;')

In [13]:
# plots the encounter multiplier vs the number of PCs for three different numbers of NPCs

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='number of PC deaths',
            range=[-0.5,8.5],
            tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier ratio',
            range=[0.95,1.85],
            tick0=0, dtick=0.1,
            tickformat='.1f',
            #minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot data
P = [2, 4, 6, 8]
for p in P:
    PD = np.arange(0, p+1)
    y = np.array([encounter_multiplier_approx(p, 1, PD=pd)/encounter_multiplier_approx(p, 1, PD=0) for pd in PD ])
    fig.add_trace(go.Scatter(
        x=PD,
        y=y,
        mode='markers+lines',
        name=f'{p:d} PCs',
        hovertemplate=
            'PCs deaths %{x:d}<br>'+
            'multiplier ratio %{y:,.2f}' + 
            '<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-single-target-em-ratio-vs-pcs-deaths-large')
    tfb.save_fig_html(fig, format='small', name='fig-single-target-em-ratio-vs-pcs-deaths-small')